# Complete Example

The subcomponents are all joined together into the main flasher class, which combines the superancillary to first check the phase, followed by 

In [ ]:
import timeit, json, functools

import teqpflsh, teqp 
import CoolProp

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
name = "n-Propane"

j = json.load(open('PROPANE_exps.json'))
sa = teqpflsh.SuperAncillary(json.dumps(j))
import CoolProp.CoolProp as CP
AS = CP.AbstractState('HEOS', 'n-Propane')
def calc(T, rho, AS, key):
    AS.specify_phase(CP.iphase_gas)
    AS.update(CP.DmolarT_INPUTS, rho, T)
    val = AS.keyed_output(key)
    AS.unspecify_phase()
    return val
sa.add_variable(k='H', caller=functools.partial(calc, AS=AS, key=CP.iHmolar))
sa.add_variable(k='S', caller=functools.partial(calc, AS=AS, key=CP.iSmolar))
sa.add_variable(k='U', caller=functools.partial(calc, AS=AS, key=CP.iUmolar))

path = f'{teqp.get_datapath()}/dev/fluids/{name}.json'
jresid = {"kind": "multifluid", "model": {"components": [name], "root": teqp.get_datapath()}}
jidealgas = {"kind": "IdealHelmholtz", "model": [teqp.convert_CoolProp_idealgas(path, 0)]}
rf = teqpflsh.RegionedFlasher(
    ideal_gas=json.dumps(jidealgas), 
    resid=json.dumps(jresid), 
    mole_fractions=np.array([1.0])
)

# Now we make a region with rectangular shape in T, rho coordinates with only its 5 corners
Tmin = 400 # K
Tmax = 450 # K
rhomin = 1e-6 # mol/m³
rhomax = 6000 # mol/m³
Tpoly = np.array([Tmin, Tmin, Tmax, Tmax, Tmin])
rhopoly = np.array([rhomin, rhomax, rhomax, rhomin, rhomin])
NT = 1000
Nrho = 1000
rf.add_region(Tpoly, rhopoly, NT=NT, Nrho=Nrho)
for reg in rf.get_regions_rw():
    reg.add_pair(proppair=teqpflsh.PropertyPairs.PS, Nsplit=5)

# Build the helper class holding the models for ideal gas and residual Helmholtz energy
helm = teqpflsh.teqpHelmholtzInterface(ideal_gas=json.dumps(jidealgas), residual=json.dumps(jresid))

# Build the main flasher which includes all the parts
mf = teqpflsh.MainFlasher(regions=rf, superancillary=sa, helm=helm)

In [ ]:
reg = mf.regioned_flasher.get_regions_ro()[0]
propset = reg.propset_Trhogrid
val1 = propset.p
val2 = propset.s + np.random.random(val1.shape)
T = np.zeros_like(val1)
rho = np.zeros_like(val1)
q = np.zeros_like(val1)

%timeit mf.flash_many(teqpflsh.PropertyPairs.PS, val1, val2, T, rho, q)